Never mind this for now, there is an error somewhere.

In [ ]:
from __future__ import print_function
from collections import defaultdict
import numpy as np
import periodictable
import matplotlib.pyplot as plt
%matplotlib inline
import chempy
from chempy.chemistry import Solute, Equilibrium
from chempy.equilibria import EqSystem
print('ChemPy: %s' % chempy.__version__)

In [ ]:
substances = [
    Solute(n, latex_name=l, composition=c) for n, l, c in [
        ('H/+', 'H^+', {0: 1, 1: 1}),
        ('OH/-', 'OH^-', {0: -1, 1: 1, 8: 1}),
        ('HSCN', 'HSCN', {1: 1, 6: 1, 7: 1, 16: 1}),
        ('SCN/-', 'SCN^-', {0: -1, 6: 1, 7: 1, 16: 1}),
        ('H2O', 'H_2O', {1: 2, 8: 1}),
        ('Fe/3+', 'Fe^{3+}', {0: 3, 26: 1}),
        ('FeSCN/2+', 'FeSCN^{2+}', {0: 2, 6: 1, 7: 1, 16: 1, 26: 1}),
        ('FeSCN2/+', 'Fe(SCN)_2^{+}', {0: 1, 6: 2, 7: 2, 16: 2, 26: 1}), 
        ('FeSCN3', 'Fe(SCN)_3', {6: 3, 7: 3, 16: 3, 26: 1}),
        ('Fe2OH2/4+', 'Fe_2(OH)_2^{4+}', {0: 4, 1: 2, 8: 2, 26: 2}),
        ('FeOH/2+', 'Fe(OH)^{2+}', {0: 2, 1: 1, 8: 1, 26: 1})]]

In [ ]:
init_conc = defaultdict(lambda: 1e-50, {'H/+': 1e-7, 'OH/-': 1e-7, 'HSCN': 1e-3, 'Fe/3+': 1e-3, 'H2O': 55.4})

In [ ]:
H2O_c = init_conc['H2O']
w_autop = Equilibrium({'H2O': 1}, {'H/+': 1, 'OH/-': 1}, 10**-14/H2O_c)
HSCN_pr = Equilibrium({'HSCN': 1}, {'H/+': 1, 'SCN/-': 1}, 10**1.28, ref='doi:10.1139/v00-152')
FeOH = Equilibrium({'Fe/3+': 1, 'H2O': 1}, {'FeOH/2+': 1, 'H/+': 1}, 10**-2.81 / H2O_c, ref='doi:10.1039/B001811M')
Fe2OH2 = Equilibrium({'Fe/3+': 2, 'H2O': 2}, {'Fe2OH2/4+': 1, 'H/+': 2}, 10**-2.81 / H2O_c**2, ref='doi:10.1039/B001811M')
FeSCN_B1 = Equilibrium({'FeSCN/2+': 1}, {'Fe/3+': 1, 'SCN/-': 1}, 10**-2.065, ref='doi:10.1039/B001811M')
FeSCN_B2 = Equilibrium({'FeSCN2/+': 1}, {'FeSCN/2+': 1, 'SCN/-': 1}, 10**-3.34, ref='doi:10.1351/pac199769071489')
FeSCN_B3 = Equilibrium({'FeSCN3': 1}, {'FeSCN2/+': 1, 'SCN/-': 1}, 10**-3.82, ref='doi:10.1351/pac199769071489')
equilibria = w_autop, HSCN_pr, FeOH, Fe2OH2, FeSCN_B1, FeSCN_B2, FeSCN_B3

In [ ]:
eqsys = chempy.equilibria.EqSystem(equilibria, substances)

In [ ]:
from chempy.util.table import rsys2pdf_table
rsys2pdf_table(eqsys, save='equilibria.pdf', siunitx=True, param_name='Equilibrium constant')
from IPython.display import HTML
HTML('<iframe src=./equilibria.pdf width=700 height=350></iframe>')

In [ ]:
HSCN_varied = np.logspace(-6, -2)
xout, info, sanity = eqsys.roots(init_conc, HSCN_varied, 'HSCN', plot_kwargs={'latex_names': True})
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.gca().set_ylim([1e-4, 1e-2])